# Serial communication on remote instruments

Some instruments support serial communication through USB or RS232 ports. While hubs can extend the number of supported devices, sometimes other limitations such as distance do not allow equipment to be physically connected to a single instrumentation server, complicating experiments.

Lightlab's solution to this is the "remote_serial" driver, a secondary client-server interface that relays commands between a user-controlled "client" machine and a remote "server" machine that is physically connected to the instrument to be controlled:

`User ---> [CLIENT e.g. lab instrumentation server] <---ZeroMQ---> [SERVER e.g. Raspberry Pi] <---Serial---> Instrument`

This is based on the asynchronous messaging platform [ZeroMQ](https://zeromq.org/). A ZMQ server is fast, lightweight, and can ensure continuity between experimental sessions.

A downside of the ZMQ method is that a server process must be initialized on the server device. This can be automated by Lightlab. When instanciating an instrument with the `remote_serial` interface, the driver first checks if the requested server is already running on the server, and if not, it uses [Fabric](https://www.fabfile.org/index.html) to upload the server code to the server and launch the server. The user only needs to ensure that the device can be found on the network accessible to the client, and note the login credentials and address.

Current requirements of the remote server:
* Python 3+
* tmux
* (only to launch the server using lightlab) passwordless SSH ability (copy-ssh-id)

## Example with MDT693B_OLPC driver

The Thorlabs MDT693B Piezo Controller has a USB interface. We want to control remotely from a central server (in this context, the "client"), so we connect it to a Raspberry PI controller (here, the "server") which is network-visible to the client.

The MDT693B_OLPC driver inherits from the ZMQclient class. Instead of a GPIB or Ethernet address, it needs the username and IP address of its controller (here, Raspberry Pi). Note that one full `zmq_timeout` is required to check if the code fails to detect a live server on the server.

In [7]:
from lightlab.equipment.lab_instruments.MDT693B_OLPC import MDT693B_OLPC
from lightlab.equipment.visa_bases import ZMQclient

import numpy as np
from IPython.display import clear_output
import serial

stage = MDT693B_OLPC(name='stage', 
                # Server SSH settings
                server_user='pi',
                server_address='128.112.50.75',
                # ZMQ settings
                zmq_port=5556,
                zmq_timeout=15,
                zmq_retries=3,
                server_filename='./tmp/serial_server.py',
                tmux_session_prefix='zmq',
                separator='___',
                # Serial equipment settings
                serial_port="/dev/ttyACM0",
                serial_baud=115200,
                serial_timeout=5,
                        )

Starting server 5556 on pi:128.112.50.75


#### Server arguments

The first arguments parametrize the `client` <--> `server` SSH connection.

`server_address` is the IP address of the server controller.

`server_user` is the username of the server controller.

As of now, this scheme requires [passwordless access](https://linuxize.com/post/how-to-setup-passwordless-ssh-login/) from the client to the server. Running `ssh server_user@server_address` should not prompt for a password.

#### ZMQ arguments

The zmq arguments parametrize the `client` <--> `server` ZeroMQ (continuous) connection.

`zmq_port` is the TCP socket that zmq will use to relay commands. If more than one such connection is made to the controller, different values should be used.

`zmq_timeout` timeout for server <--> client communication

`zmq_retries` is how many reconnection attempts are performed when a command cannot reach the server.

`server_filename` is the filename under which the server script is uploaded to the server. Default is `~/tmp/serial_server.py`

`tmux_session_prefix` is the prefix of the tmux session under which the zmq server is persistently run after severing the SSH connection. By default, it is `zmq`, and results in a full session name `{tmux_session_prefix}_{zmq_port}`.

`separator` is the delimiting character(s) that separate the command header from the command. By default `___`.


#### Serial connection arguments

The below arguments parametrize the `server` <--> `instrument` connection.

The `serial_baud` was obtained from the instrument's manual.

The `serial_port` was found by scanning the controller's USB ports. The below bash script may help:

```
#!/bin/bash

for sysdevpath in $(find /sys/bus/usb/devices/usb*/ -name dev); do
    (
        syspath="${sysdevpath%/dev}"
        devname="$(udevadm info -q name -p $syspath)"
        [[ "$devname" == "bus/"* ]] && exit
        eval "$(udevadm info -q property --export -p $syspath)"
        [[ -z "$ID_SERIAL" ]] && exit
        echo "/dev/$devname - $ID_SERIAL"
    )
done
```

The timeouts are made high enough to ensure reliable communication (trial and error might be required here).

### Driver-level operation

Since MDT693B_OLPC inherits from the ZMQclient class, its driver functions can directly call the `request`, `write`, etc. method. Hence, a programmer only needs to translate serial commands from the MDT693B like so:

```
def get_volt(self, axis):
    cmd = "{}voltage?".format(axis)
    ans = self.request(cmd)
    return float(ans.strip('[]\r> '))
```

Here, the `cmd` to read the voltage was lifted from the instrument programming manual. The `request` method from `ZMQclient`, via the instanciated `ZMQserver` instanciated on the controller, will pass the string command to the instrument and return the result. The user then only needs to parse as required by the instrument response. There is also a `write` method which sends messages to the instrument without waiting for a response.

In [8]:
stage.get_volt('x')

12.05

In [9]:
stage.set_volt('x', 15)

In [10]:
stage.get_volt('x')

15.04

In [11]:
stage.terminate()

1

Since we have terminated the server (which should not really be required), below is a timeout (need to improve error handling):

In [12]:
stage.get_volt('x')

ZMQError: Operation not supported

### Under the hood

The ZMQ client <--> server purely exchanges messages, and so details are left to the programmer. Here, the `client` attempts to send messages to a continuously-listening `server` (through its `run` method), and waits for a response. Some amount of message parsing is done at the `server` level through alphanumeric `cmd_header` values appended at the beginning of the communication, parsed through the `separator` argument. This allows some commands to not be relayed to the connected instrument and trigger something on the server itself (such as `ping`, `terminate`, etc.), or to alter the behaviour of the serial communication (for instance, `request` having along timeout to wait for responses, and `write` having a short one). There may be more robust ways to do this, for instance with sequences of messages.